In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import  OrdinalEncoder
import joblib
import sys
import datetime
from sklearn.cluster import DBSCAN
import re
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import wordnet, pos_tag
from nltk import WordNetLemmatizer
nltk.download('stopwords')
sw_eng = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
!pip install catboost
from google.colab import files
from catboost import CatBoostClassifier 
pd.set_option('use_inf_as_na', True)
np.random.seed(42)

nltk.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


'3.7'

In [ ]:
def delete_stop_words(doc):
  sw_eng = set(stopwords.words('english'))
  return ' '.join([word for word in doc.split() if not (word in sw_eng)])

def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN


def my_lemmatizer(sent):
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])
      
def preprocess_data(data):
  X = data.copy()

  X['Count photos'] = X.photos.map(lambda s: s.count('http'))

  X['latitude'] = X['latitude'].replace(0,np.NaN)
  X['longitude'] = X['longitude'].replace(0,np.NaN)

  X['long*lat'] = X['longitude'] * X['latitude']

  clustering = DBSCAN(eps=1.01e-4, min_samples=1).fit(X[['latitude', 'longitude']].fillna(0))
  X['new id'] = pd.Series(clustering.labels_)
  
  X['bath*bed'] = X['bedrooms'] * X['bathrooms']
  X['bath/bed'] = X['bathrooms'] / X['bedrooms']
  X['bath/bed'] = X['bath/bed'].replace(np.inf, 0)
  X['bath/bed'].where(X['bath/bed'] != 0, other=X['bathrooms']*2, inplace=True)

  X['price*bed'] = X['price'] * X['bedrooms']
  X['price*bath'] = X['price'] * X['bathrooms']

  X['price/bed'] = X['price'] / X['bedrooms']  
  X['price/bath'] = X['price'] / X['bathrooms']

  X['price/bed'] = X['price/bed'].replace(np.inf, 0)
  X['price/bed'].where(X['price/bed'] != 0, other=X['price']*2, inplace=True)
  X['price/bath'] = X['price/bath'].replace(np.inf, 0)
  X['price/bath'].where(X['price/bath'] != 0, other=X['price']*2, inplace=True)

  X['price/(bedrooms+bathrooms)'] = X['price'] / (X['bedrooms'] + X['bathrooms'])
  X['price/(bedrooms+bathrooms)'] = X['price/(bedrooms+bathrooms)'].replace(np.inf, 0)
  X['price/(bedrooms+bathrooms)'].where(X['price/(bedrooms+bathrooms)'] != 0, other=X['price']*2, inplace=True)

  X['price*count photos'] = X['price'] * X['Count photos']

  X['price/count photos'] = X['price'] / X['Count photos']
  X['price/count photos'] = X['price/count photos'].replace(np.inf, 0)
  X['price/count photos'].where(X['price/count photos'] != 0, other=X['price']*2, inplace=True)

  X['Created month'] = pd.to_datetime(X['created']).map(lambda x: x.month)

  X['price*Created month'] = X['price'] * X['Created month']
  X['price/Created month'] = X['price'] / X['Created month']

  X['count photos*Created month'] = X['Count photos'] * X['Created month']
  X['count photos/Created month'] = X['Count photos'] / X['Created month']

  X['display_address'] = data.display_address.astype(str).map(lambda s: re.sub('[^ a-z0-9]', "", (' ' + s + ' ').lower())
                                                                .replace(' w ', ' west ').replace(' e ', ' east ').replace(' st ', ' street ').replace(' ave ', ' avenue ')
                                                                .replace('first', '1').replace('second', '2').replace('third', '3').replace('fourth', '4')
                                                                .replace('fifth', '5').replace('sixth', '6').replace('seventh', '7').replace('eighth', '8')
                                                                .replace('ninth', '9').replace('tenth', '10')
                                                                .replace('2nd', '2').replace('1st', '1').replace('3rd', '3').replace('th', '')
                                                                .strip())
  
  X['display_address'] = X['display_address'].map(lambda s: re.sub('^\d+', "", s).strip() if (len(re.findall('\d+', s)) == 2) else s)

  X['features'] = data.features.map(lambda s: re.sub('[^a-zA-Z ]', " ", s.lower())) + ' ' + data.description.astype(str).map(lambda s: re.sub('[^a-zA-Z ]', " ", s.lower())) 
  X['features'] = X['features'].map(lambda s: delete_stop_words(s.strip()))
  X['features'] = X['features'].map(lambda s: my_lemmatizer(s))

  # categotical features for catboost
  X['building_id'] = X['building_id'].astype(str)
  X['manager_id'] = X['manager_id'].astype(str)
  X['bathrooms'] = X['bathrooms'].astype(str)
  X['bedrooms'] = X['bedrooms'].astype(str)
  X = X.drop(['description', 'photos', 'street_address', 'created', 'Id', 'listing_id'], axis=1) 
  return X

In [ ]:
test_data = pd.read_csv('test.csv')
train_data =  pd.read_csv('train.csv')
train_data

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,TARGET
0,57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
4,592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,25582,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,2016-06-16 02:12:57,Newly renovated bedroom apartment located off ...,29th St,['No pets'],40.7653,7166867,-73.9248,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7166867_625fde3...,1950,30-95 29th St,low
34542,50013,1.0,0,9b6cf886379a2511f8c633c84028efe7,2016-05-10 03:17:32,All apartments are newly renovated featuring: ...,E 1st St,"['Pre-War', 'Laundry in Unit', 'Dishwasher', '...",40.7239,6989251,-73.9901,5599e962719af3ccc2976855c2d5893c,['https://photos.renthop.com/2/6989251_42a4884...,2600,39 E 1st St,medium
34543,111475,2.0,2,0,2016-04-21 03:29:35,"<![CDATA[2 bedrooms, 5110, Astoria / Long Isla...",50th Avenue,[],40.7437,6905568,-73.9585,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/6905568_01602e3...,5110,2-01 50th Avenue,low
34544,71184,1.0,2,8754cae39f6e053974aa2337017eb3c1,2016-05-14 02:27:47,CooperCooper.com :: Listing ID #10_0385; 400 W...,400 West 56th Street,"['Laundry in Unit', 'Dishwasher', 'Cats Allowe...",40.7673,7010187,-73.9866,6e5c10246156ae5bdcd9b487ca99d96a,['https://photos.renthop.com/2/7010187_ab1db5d...,3400,400 West 56th Street,medium


In [ ]:
df = preprocess_data(train_data)
y_train = df['TARGET']
X_train = df.drop(['TARGET'], axis=1)
X_test = preprocess_data(test_data)
X_train

,bathrooms,bedrooms,building_id,display_address,features,latitude,longitude,manager_id,price,Count photos,long*lat,new id,bath*bed,bath/bed,price*bed,price*bath,price/bed,price/bath,price/(bedrooms+bathrooms),price*count photos,price/count photos,Created month,price*Created month,price/Created month,count photos*Created month,count photos/Created month
0,1.0,3,0,how amazing is is midtown west steal no fee,laundry unit fee elevator fabulous br midtown ...,40.7647,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,4495,3,-3016.253529,0,3.0,0.333333,13485,4495.0,1498.333333,4495.0,1123.750000,13485,1498.333333,5,22475,899.000000,15,0.600000
1,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,55 river drive sou,dog allow cat allow fee renovate kitchen bathroom,40.7275,-74.0322,e5808a5e6cc13988fe596704428d38d5,2570,13,-3015.146426,1,1.0,1.000000,2570,2570.0,2570.000000,2570.0,1285.000000,33410,197.692308,6,15420,428.333333,78,2.166667
2,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,west 77 street,elevator hardwood floor rare best deal market ...,40.7798,-73.9751,d69d4e111612dd12ef864031c1148543,1795,6,-3016.689783,2,0.0,2.000000,0,1795.0,3590.000000,1795.0,1795.000000,10770,299.166667,5,8975,359.000000,30,1.200000
3,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,john street,swim pool doorman elevator fitness center laun...,40.7081,-74.0065,e6472c7237327dd3903b3d6f6a94515a,3400,4,-3012.664003,3,2.0,0.500000,6800,3400.0,1700.000000,3400.0,1133.333333,13600,850.000000,5,17000,680.000000,20,0.800000
4,1.0,3,fee4d465932160318364d9d48d272879,west 16 street,laundry build laundry unit dishwasher hardwood...,40.7416,-74.0025,6fba9b3a8327c607b8b043716efee684,5695,3,-3014.980254,4,3.0,0.333333,17085,5695.0,1898.333333,5695.0,1423.750000,17085,1898.333333,6,34170,949.166667,18,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,29 street,pet newly renovate bedroom apartment locate th...,40.7653,-73.9248,e6472c7237327dd3903b3d6f6a94515a,1950,4,-3013.566649,6096,1.0,1.000000,1950,1950.0,1950.000000,1950.0,975.000000,7800,487.500000,6,11700,325.000000,24,0.666667
34542,1.0,0,9b6cf886379a2511f8c633c84028efe7,east 1 street,pre war laundry unit dishwasher hardwood floor...,40.7239,-73.9901,5599e962719af3ccc2976855c2d5893c,2600,6,-3013.165433,2521,0.0,2.000000,0,2600.0,5200.000000,2600.0,2600.000000,15600,433.333333,5,13000,520.000000,30,1.200000
34543,2.0,2,0,50 avenue,cdata bedroom astoria long island city br br l...,40.7437,-73.9585,e6472c7237327dd3903b3d6f6a94515a,5110,15,-3013.342936,1377,4.0,1.000000,10220,10220.0,2555.000000,2555.0,1277.500000,76650,340.666667,4,20440,1277.500000,60,3.750000
34544,1.0,2,8754cae39f6e053974aa2337017eb3c1,west 56 street,laundry unit dishwasher cat allow dog allow co...,40.7673,-73.9866,6e5c10246156ae5bdcd9b487ca99d96a,3400,3,-3016.233918,3846,2.0,0.500000,6800,3400.0,1700.000000,3400.0,1133.333333,10200,1133.333333,5,17000,680.000000,15,0.600000


In [ ]:
cbc = CatBoostClassifier(logging_level = 'Silent', eval_metric = 'MultiClass',
                         cat_features = ['bathrooms', 'bedrooms', 'building_id', 'new id', 'manager_id','Count photos'],
                         text_features = ['display_address',	'features'],
                         auto_class_weights='Balanced', 
                         od_wait = 200, od_type='Iter')

param_grid = {
    'iterations': [3000], 
    'learning_rate': [0.2, 0.17, 0.15, 0.13, 0.11, 0.1, 0.09, 0.07, 0.05],
    'depth': [3],  
    'l2_leaf_reg': [3, 5, 7, 9],
    'random_strength' :[50, 100, 150, 200, 250],
    'border_count':[254]
}



res = cbc.grid_search(param_grid, X_train, y_train, verbose=False, refit= True, cv = 5, partition_random_seed=42)

In [ ]:
print(cbc.best_score_)
res.get('params')

{'learn': {'MultiClass': 0.5491314431549824}}


{'border_count': 254,
 'depth': 3,
 'iterations': 3000,
 'l2_leaf_reg': 3,
 'learning_rate': 0.15,
 'random_strength': 250}

In [ ]:
y_pred = cbc.predict(X_test)
y_pred = y_pred.ravel()
pd.DataFrame({'Id': test_data['Id'], 'TARGET' : y_pred}).to_csv('my_submission.csv', index = False)
files.download('my_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>